In [5]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

import tweepy
import csv
import json
import pandas as pd
import os

In [7]:
# Twitter API credentials
credentialsPath = r'..\0_data\credentials'
with open(os.path.join(credentialsPath, 'twitter_credentials.json')) as cred_data:
    info = json.load(cred_data)
    consumer_key = info['CONSUMER_KEY']
    consumer_secret = info['CONSUMER_SECRET']
    access_key = info['ACCESS_KEY']
    access_secret = info['ACCESS_SECRET']

# Create the api endpoint

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
api = tweepy.API(auth)

In [9]:
# Convert to data frame
def toDataFrame(tweets):
    DataSet = pd.DataFrame()
    
    tweetsText = []
    for tweet in tweets:
            if 'retweeted_status' in  dir(tweet):
                tweetsText.append(tweet.retweeted_status.full_text.encode('utf-8'))                
            else:
                tweetsText.append(tweet.full_text.encode('utf-8'))
            
    DataSet['Text'] = [text for text in tweetsText]
    DataSet['User'] = [tweet.user.name.encode('utf-8') for tweet in tweets]
    
    tweetsImages = []
    for tweet in tweets:
        if 'media' in tweet.entities:
            for image in tweet.entities['media']:
                tweetsImages.append(image['media_url'])
        else:
            tweetsImages.append('')
            
    
    DataSet['Image Urls'] = [image for image in tweetsImages]
            
    tweetsLongitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLongitudes.append(tweet.coordinates["coordinates"][0])
        else:
            tweetsLongitudes.append('')
    DataSet['Longitude'] = [longitude for longitude in tweetsLongitudes]
    
    tweetsLatitudes = []
    for tweet in tweets:
        if tweet.coordinates is not None:
            tweetsLatitudes.append(tweet.coordinates["coordinates"][1])
        else:
            tweetsLatitudes.append('')
    DataSet['Latitude'] = [latitude for latitude in tweetsLatitudes]
    
    return DataSet


# Specify the maximum number of tweets that you want to be extracted.

maximum_number_of_tweets_to_be_extracted = \
    int(input('Enter the number of tweets that you want to extract- '))

# Mention the mention that you want to look out for

mention = input('Enter the mention you want to scrape- ')

results = []

for tweet_info in tweepy.Cursor(api.search, q='@' + mention,
                           tweet_mode='extended').items(maximum_number_of_tweets_to_be_extracted):
    results.append(tweet_info)
    
data = toDataFrame(results)
outputPath = r'..\0_data\manual'
filePath = os.path.join(outputPath,'tweets_with_mention_' + mention + '.csv')
if not os.path.isfile(filePath):
    data.to_csv(filePath, index=False)
else:
    with open(filePath, 'a') as file:
        data.to_csv(file, index = False)
print ('Extracted ' + str(maximum_number_of_tweets_to_be_extracted) \
    + ' tweets with mention @' + mention)

Enter the number of tweets that you want to extract- 10
Enter the mention you want to scrape- sfmta_muni
Extracted 10 tweets with mention @sfmta_muni
